In [1]:
# The magic commands below allow reflecting the changes in an imported module without restarting the kernel.
%load_ext autoreload
%autoreload 2

# We need to add balsam and the modules it depends on to the Python search paths. 
import sys
sys.path.insert(0,'/soft/datascience/Balsam/0.3.5.1/env/lib/python3.6/site-packages/')
sys.path.insert(0,'/soft/datascience/Balsam/0.3.5.1/')
from pprint import pprint
pprint(f'Python search path includes: {sys.path}')

# We also need postgresql to be in the path
import os
os.environ['PATH'] ='/soft/datascience/Balsam/0.3.5.1/env/bin/:' + os.environ['PATH']
os.environ['PATH'] +=':/soft/datascience/PostgreSQL/9.6.12/bin/'

try:
    import balsam
except:
    print('Cannot find balsam, make sure balsam is installed or it is available in Python search paths')    

    # We also need to activate Balsam database by setting the BALSAM_DB_PATH environment variable. 
# This is equivalent to `source balsamactivate jupyter_test` 
#os.environ["BALSAM_DB_PATH"]='/lus/theta-fs0/projects/datascience/keceli/balsam/jupyter_test'

("Python search path includes: ['/soft/datascience/Balsam/0.3.5.1/', "
 "'/soft/datascience/Balsam/0.3.5.1/env/lib/python3.6/site-packages/', '', "
 "'/opt/anaconda3/lib/python36.zip', '/opt/anaconda3/lib/python3.6', "
 "'/opt/anaconda3/lib/python3.6/lib-dynload', "
 "'/opt/anaconda3/lib/python3.6/site-packages', "
 "'/opt/anaconda3/lib/python3.6/site-packages/Mako-1.0.7-py3.6.egg', "
 "'/opt/anaconda3/lib/python3.6/site-packages/IPython/extensions', "
 "'/gpfs/mira-home/keceli/.ipython']")


In [2]:
def get_database_paths(verbose=True):
    """
    Prints the paths for existing balsam databases
    """
    try:
        from balsam.django_config.db_index import refresh_db_index
        databasepaths = refresh_db_index()
    except:
        databasepaths = None
    if verbose:
        if len(databasepaths) > 0:
            print(f'Found {len(databasepaths)} balsam database location')
            for db in databasepaths:
                print(db)
        else:
            print('No balsam database found')
    return databasepaths

def get_active_database(verbose=True):
    """
    Gets the activate database set in environment variable BALSAM_DB_PATH
    Parameters:
    verbose: Boolean, (True): Prints verbose info (False): No print
    Returns
    -------
    str, path for the active database
    """
    try:
        db = os.environ["BALSAM_DB_PATH"]
        if verbose: print(f'Active balsam database path: {db}')
    except:
        if verbose: print('BALSAM_DB_PATH is not set')
        db = None
    return db
    
def add_app(name, executable, description='', envscript='', preprocess='', postprocess='', checkexe=False):
    """
    Adds a new app to the balsam database.
    """
    from balsam.core.models import ApplicationDefinition as App
    import shutil
    newapp = App()
    if checkexe:
        if shutil.which(executable):        
            print('{} is found'.format(executable))
        else:
            print('{} is not found'.format(executable))
            return newapp
        
    if App.objects.filter(name=name).exists():
        print("An application named {} already exists".format(name))
    else:
        newapp.name        = name
        newapp.executable  = executable
        newapp.description = description
        newapp.envscript   = envscript
        newapp.preprocess  = preprocess
        newapp.postprocess = postprocess
        newapp.save()
    return newapp

def get_apps(verbose=True):
    """
    Returns all apps as a list
    """
    try:
        from balsam.core.models import ApplicationDefinition as App
        apps = App.objects.all()
    except:
        apps = None
    return apps
        
def get_job():
    from balsam.launcher.dag import BalsamJob
    return BalsamJob()

def add_job(name, workflow, application, description='', args='', num_nodes=1, ranks_per_node=1,cpu_affinity='depth',data={},environ_vars={}):
    from balsam.launcher.dag import BalsamJob
    job                = BalsamJob()
    job.name           = name
    job.workflow       = workflow
    job.application    = application
    job.description    = description
    job.args           = args
    job.num_nodes      = num_nodes
    job.ranks_per_node = ranks_per_node
    job.cpu_affinity   = cpu_affinity
    job.environ_vars   = environ_vars
    job.data           = {}
    job.save()
    
def submit(project='datascience',queue='debug-flat-quad',nodes=1,wall_minutes=30,job_mode='mpi',wf_filter=''):
    from balsam import setup
    setup()
    from balsam.service import service
    from balsam.core import models
    QueuedLaunch = models.QueuedLaunch
    mylaunch = QueuedLaunch()
    mylaunch.project = project
    mylaunch.queue = queue
    mylaunch.nodes = nodes
    mylaunch.wall_minutes = wall_minutes
    mylaunch.job_mode = job_mode
    mylaunch.wf_filter = wf_filter
    mylaunch.prescheduled_only=False
    mylaunch.save()
    service.submit_qlaunch(mylaunch, verbose=True)

In [23]:
# Add an app that executes `sleep` command, which pauses for a given number of seconds.
# If app already exists, you don't need to do anything
add_app(name='sleep',executable='sleep',description='Pause for a given number of seconds, see `sleep --help` linux command')

An application named sleep already exists


Application None:
-----------------------
name:                           
description:                    
executable:                     
preprocess:                     
envscript:                      
postprocess:                    

In [18]:
# A simple test for adding 5 jobs that  pauses 0, 10, 20, 30, 40 seconds
for i in range(5):
    t = i*10
    add_job(name=f'sleep_{t}',workflow='sleep_test1',application='sleep',args=f'{t}')
    

In [25]:
# If you see 'Submit OK:', Job submission is succesful.
submit()

Submit OK: Qlaunch {   'command': '/lus/theta-fs0/projects/datascience/keceli/balsam/jupyter_test/qsubmit/qlaunch4.sh',
    'from_balsam': True,
    'id': 4,
    'job_mode': 'mpi',
    'nodes': 1,
    'prescheduled_only': False,
    'project': 'datascience',
    'queue': 'debug-flat-quad',
    'scheduler_id': 352681,
    'state': 'submitted',
    'wall_minutes': 25,
    'wf_filter': ''}


In [3]:
# You can check run times after job is completed. 
# You may see ~ 5 - 10 seconds longer runtimes than the sleep times due to initializations.
from balsam.launcher.dag import BalsamJob
for j in BalsamJob.objects.filter(state='JOB_FINISHED').all():
    print(j.name,j.runtime_seconds)

mpibench_32 20.882527
sleep_22 27.883907
sleep_33 38.882066
sleep_44 49.883403
sleep_10 13.882113
sleep_20 25.88482
sleep_40 45.881849
sleep_0 5.888986
sleep_30 35.886307
mpibench_4 7.881661
sleep_0 3.88008
mpibench_8 11.885939
sleep_11 16.8839
mpibench_16 14.882554
mpibench_1 6.881617
mpibench_2 8.882998


In [4]:
for j in BalsamJob.objects.all():
    print(j.name,j.state)

mpibench_32 JOB_FINISHED
sleep_22 JOB_FINISHED
sleep_33 JOB_FINISHED
sleep_44 JOB_FINISHED
sleep_10 JOB_FINISHED
sleep_20 JOB_FINISHED
sleep_40 JOB_FINISHED
mpibench_64 PREPROCESSED
sleep_0 JOB_FINISHED
mpibench_64 PREPROCESSED
sleep_30 JOB_FINISHED
mpibench_64 PREPROCESSED
mpibench_4 JOB_FINISHED
sleep_0 JOB_FINISHED
mpibench_8 JOB_FINISHED
sleep_11 JOB_FINISHED
mpibench_16 JOB_FINISHED
mpibench_1 JOB_FINISHED
mpibench_64 PREPROCESSED
mpibench_64 PREPROCESSED
mpibench_64 PREPROCESSED
mpibench_2 JOB_FINISHED
